In [ ]:
#Import data

In [34]:
from __future__ import print_function  # Python 2 and 3
from sklearn.preprocessing import StandardScaler
import pandas as pd
import scipy.stats
import numpy as np
from numpy.random import shuffle
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn import linear_model
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LassoCV, ElasticNetCV
import xgboost as xgb
from sklearn.svm import SVR
#from scipy import stats
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
plt.style.use('ggplot')
import statsmodels.api as sm 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import decimal
import sklearn.feature_selection as fs

# Read in Training Set & Filter For Variance

In [73]:
categorical_columns = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
                       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
                      'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 
                      'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
                       'MasVnrType',
                       'Foundation', 'BsmtFinType1', 'BsmtFinType2',
                       'Heating', 'CentralAir', 'Electrical', 'Functional', 
                       'GarageType', 'GarageFinish', 'PavedDrive', 'Fence',
                       'MiscFeature', 'SaleType', 'SaleCondition']
drops = ['GarageArea', 'GarageYrBlt', 'Id', 'FullBath',
         'TotRmsAbvGrd', 'X2ndFlrSF', 'BsmtFullBath',
         'MSZoning', 'SalePrice', ]
categorical_columns = list(filter(lambda v: v not in drops, categorical_columns))

In [74]:
df = pd.read_csv('post_process_train.csv', index_col = 0)

y = list(df['SalePrice'])
df = df.drop(drops, axis = 1)
pd.set_option('display.max_columns', None)
df_new = pd.concat([pd.DataFrame(fs.VarianceThreshold(threshold = .1).fit_transform(df.drop(categorical_columns, axis=1))), df[categorical_columns]], axis=1)
df_new = df_new.drop(0, axis = 0)
del y[0]
df_new = df_new.drop(len(df), axis = 0)
df_new

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,MSSubClass,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,Foundation,BsmtFinType1,BsmtFinType2,Heating,CentralAir,Electrical,Functional,GarageType,GarageFinish,PavedDrive,SaleType,SaleCondition
1,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,4.0,3.0,4.0,3.0,2.0,486.0,0.0,434.0,920.0,5.0,920.0,0.0,1786.0,1.0,3.0,4.0,1.0,3.0,2.0,3.0,3.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,60.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,WD,Normal
2,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,3.0,3.0,3.0,4.0,1.0,216.0,0.0,540.0,756.0,4.0,961.0,0.0,1717.0,0.0,3.0,4.0,1.0,4.0,3.0,3.0,3.0,0.0,35.0,272.0,0.0,0.0,0.0,0.0,2.0,2006.0,60.0,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,WD,Normal
3,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,4.0,3.0,4.0,3.0,3.0,655.0,0.0,490.0,1145.0,5.0,1145.0,0.0,2198.0,1.0,4.0,4.0,1.0,3.0,3.0,3.0,3.0,192.0,84.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,70.0,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,None,BrkTil,ALQ,Unf,GasA,Y,SBrkr,Typ,Detchd,Unf,Y,WD,Abnorml
4,85.0,14115.0,5.0,5.0,1993.0,1995.0,0.0,3.0,3.0,4.0,3.0,1.0,732.0,0.0,64.0,796.0,5.0,796.0,0.0,1362.0,1.0,1.0,3.0,0.0,0.0,2.0,3.0,3.0,40.0,30.0,0.0,320.0,0.0,0.0,700.0,10.0,2009.0,60.0,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,WD,Normal
5,75.0,10084.0,8.0,5.0,2004.0,2005.0,186.0,4.0,3.0,5.0,3.0,3.0,1369.0,0.0,317.0,1686.0,5.0,1694.0,0.0,1694.0,0.0,3.0,4.0,1.0,4.0,2.0,3.0,3.0,255.0,57.0,0.0,0.0,0.0,0.0,0.0,8.0,2007.0,50.0,Pave,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,Gable,CompShg,VinylSd,VinylSd,None,Wood,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,Unf,Y,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1410,62.0,7917.0,6.0,5.0,1999.0,2000.0,0.0,3.0,3.0,4.0,3.0,1.0,0.0,0.0,953.0,953.0,5.0,953.0,0.0,1647.0,1.0,3.0,3.0,1.0,3.0,2.0,3.0,3.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,8.0,2007.0,20.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,PConc,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,WD,Normal
1411,85.0,13175.0,6.0,6.0,1978.0,1988.0,119.0,3.0,3.0,4.0,3.0,1.0,790.0,163.0,589.0,1542.0,3.0,2073.0,0.0,2073.0,0.0,3.0,3.0,2.0,3.0,2.0,3.0,3.0,349.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2010.0,60.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,PConc,Unf,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,WD,Normal
1412,66.0,9042.0,7.0,9.0,1941.0,2006.0,0.0,5.0,4.0,3.0,4.0,1.0,275.0,0.0,877.0,1152.0,5.0,1188.0,0.0,2340.0,0.0,4.0,4.0,2.0,4.0,1.0,3.0,3.0,0.0,60.0,0.0,0.0,0.0,0.0,2500.0,5.0,2010.0,20.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,Plywood,Plywood,Stone,CBlock,ALQ,Rec,GasA,Y,SBrkr,Min1,Attchd,Unf,Y,WD,Normal
1413,68.0,9717.0,5.0,6.0,1950.0,1996.0,0.0,3.0,3.0,3.0,3.0,2.0,49.0,1029.0,0.0,1078.0,4.0,1078.0,0.0,1078.0,0.0,2.0,4.0,0.0,0.0,1.0,3.0,3.0,366.0,0.0,112.0,0.0,0.0,0.0,0.0,4.0,2010.0,70.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,CemntBd,CmentBd,None,Stone,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,WD,Normal


# Box Cox Transformation

In [37]:
y = scipy.stats.boxcox(y, .25)
pd.set_option('display.max_columns', None)
df.head()

,MSSubClass,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,X1stFlrSF,LowQualFinSF,GrLivArea,BsmtHalfBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,Functional,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,X3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1,60,65,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196,4,3,PConc,4,3,1,GLQ,706,Unf,0,150,856,GasA,5,Y,SBrkr,856,0,1710,0,1,3,1,4,Typ,0,0,Attchd,RFn,2,3,3,Y,0,61,0,0,0,0,0,2,2008,WD,Normal
2,60,68,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162,4,3,PConc,4,3,2,GLQ,486,Unf,0,434,920,GasA,5,Y,SBrkr,920,0,1786,0,1,3,1,4,Typ,1,3,Attchd,RFn,2,3,3,Y,0,42,0,0,0,0,0,9,2008,WD,Normal
3,70,60,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0,3,3,BrkTil,3,4,1,ALQ,216,Unf,0,540,756,GasA,4,Y,SBrkr,961,0,1717,0,0,3,1,4,Typ,1,4,Detchd,Unf,3,3,3,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml
4,60,84,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350,4,3,PConc,4,3,3,GLQ,655,Unf,0,490,1145,GasA,5,Y,SBrkr,1145,0,2198,0,1,4,1,4,Typ,1,3,Attchd,RFn,3,3,3,Y,192,84,0,0,0,0,0,12,2008,WD,Normal
5,50,85,14115,Pave,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0,3,3,Wood,4,3,1,GLQ,732,Unf,0,64,796,GasA,5,Y,SBrkr,796,0,1362,0,1,1,1,3,Typ,0,0,Attchd,Unf,2,3,3,Y,40,30,0,320,0,0,700,10,2009,WD,Normal


# Read in Kaggle Data

In [ ]:
df_kaggle = pd.read_csv('post_process_test.csv', index_col = 0).dropna()
df_kaggle = df_kaggle.drop(drops, axis = 1)

In [ ]:
len(df_kaggle.columns)

# Train/Test Split

In [75]:
df_dum = pd.get_dummies(data=df_new,columns = categorical_columns)
df_dum.shape

(1414, 242)

In [76]:
X_train, X_test, y_train, y_test = train_test_split(df_dum, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
df_kaggle.shape

##AIC & BIC

In [ ]:
X_add_const = sm.add_constant(X_train)
ols = sm.OLS(y_train, X_add_const)
ans = ols.fit()
print(ans.summary())

#Multiple Linear Regression

In [77]:
#sc = StandardScaler().fit(X_train)
#X_train = sc.transform(X_train)
#X_test = sc.transform(X_test)

mlr = linear_model.LinearRegression()
mlr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [78]:
mlr.score(X_train, y_train)

0.9101054252946228

In [79]:
mlr.score(X_test, y_test)

0.8025096910095841

In [ ]:
mlr.predict(df_kaggle)

In [ ]:
#Lasso Regression

In [43]:
lasso = Lasso()
lasso.fit(X_train, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [44]:
lasso.score(X_train, y_train)

0.8596462195175395

In [45]:
lasso.score(X_test, y_test)

0.7879820276554107

In [ ]:
lasso.predict(df_kaggle)

# Random Forest

In [46]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 100)
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [47]:
rf.score(X_train, y_train)

0.9842544568597508

In [48]:
rf.score(X_test, y_test)

0.8572208875026589

In [ ]:
rf.predict(df_kaggle)

# Beginning of XGBoost

In [118]:
mod = xgb.XGBRegressor(
    gamma=1,                 
    learning_rate=0.01,
    max_depth=3,
    n_estimators=10000,                                                                    
    subsample=0.8,
    random_state=34
) 

mod.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=1, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10000, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=34, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [119]:
mod.score(X_train,y_train)

0.9987019905976698

In [51]:
mod.score(X_test,y_test)

0.8716655795926792

In [ ]:
mod.predict(df_kaggle)

# SVR

In [52]:
svr = SVR()
svr.fit(X_train,y_train)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [53]:
svr.score(X_train,y_train)

0.4212851930282788

In [54]:
svr.score(X_test,y_test)

0.4232329491266775

# Models w/ Grid Search

In [55]:
mlrCV = linear_model.LinearRegression()
mlr_grid_param = [{'copy_X':[True,False], 'fit_intercept': [True,False], 'n_jobs': ['None'], 'normalize':[True,False]}]
para_search = GridSearchCV(estimator=mlrCV, param_grid=mlr_grid_param, scoring='r2', cv=5, return_train_score=True)
para_search = para_search.fit(X_train, y_train)

In [56]:
print(list(para_search.cv_results_.keys()))
para_search.cv_results_
print(para_search.best_score_)
print(para_search.best_params_)
para_search.best_estimator_

['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_copy_X', 'param_fit_intercept', 'param_n_jobs', 'param_normalize', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'split3_train_score', 'split4_train_score', 'mean_train_score', 'std_train_score']
0.8702473329213344
{'copy_X': True, 'fit_intercept': True, 'n_jobs': 'None', 'normalize': False}


LinearRegression(copy_X=True, fit_intercept=True, n_jobs='None',
                 normalize=False)

In [57]:
mlr_best = para_search.best_estimator_
mlr_best.fit(X_train, y_train)
mlr_best.score(X_train, y_train)

0.9243457342507964

In [58]:
mlr_best.score(X_test, y_test)

0.8345965241069445

In [59]:
lasso = LassoCV(alphas=np.linspace(0.00001,1,1000), cv=10,tol = 0.00001,max_iter = 10000)

In [60]:
L=lasso.fit(X_train, y_train)
print('train:',lasso.score(X_train, y_train),'  test:',lasso.score(X_test, y_test))

C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.314464082531231, tolerance: 0.6283512471386918
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.3447213734580146, tolerance: 0.6307457283292272
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 24.261777375009842, tolerance: 0.6270559013699766
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You

train: 0.9101234582215814   test: 0.8689068766463907


# Grid Search Random Forest

In [130]:
grid_para_tree = [{
    "criterion": ["mse"],
    "max_depth": np.linspace(start = 1, stop = 15, num = 4, dtype = int ),
    "min_samples_split": np.linspace(start=2, stop=30, num=5, dtype=int),
    "n_estimators": np.linspace(start = 80, stop = 120, num = 8, dtype = int )
}]
#tree_model.set_params(random_state=34)
grid_search_rf = GridSearchCV(rf, grid_para_tree, cv=5, scoring='r2', n_jobs=-1)
grid_search_rf = para_search.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
grid_search_rf.score(X_train, y_train)
#Make your grid coarse
#Run your first parameter, then creep up the workload
#Bayesian Hyperparameter Tuning

In [ ]:
grid_search_rf.score(X_test, y_test)

# Now XG Boost w/ CV

In [167]:
#Inspiration for this approach from https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

In [168]:
cv_params = xgb.cv(
    params,
    dtrain,
    num_boost_round=999,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)
cv_params


[21:14:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:14:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:14:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:14:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:14:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


,train-mae-mean,train-mae-std,test-mae-mean,test-mae-std
0,125853.718750,347.393391,125626.776563,633.710835
1,88897.579688,221.673873,89088.392188,692.878312
2,62929.852344,156.033490,63645.963281,1111.156725
3,44650.728125,125.635738,46361.266406,1163.279967
4,31934.953906,97.490508,34916.997656,1339.136896
5,23140.086719,64.434996,27714.166016,1131.976274
6,17195.913672,118.552248,23359.903906,1040.550912
7,13312.166016,204.719436,20826.105859,889.992991
8,10784.021875,186.576097,19490.216797,779.865128
9,9119.636914,128.258870,18821.358594,709.511911


In [169]:
min_mae = float("Inf")
best_params = None
for max_depth in range(9,12):
    params['max_depth'] = max_depth
    # Run CV
    cv_params = xgb.cv(
        params,
        dtrain,
        num_boost_round=999,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_params['test-mae-mean'].min()
    boost_rounds = cv_params['test-mae-mean'].argmin()
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth)
print("Best params: {}, MAE: {}".format(best_params, min_mae))

[21:14:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:14:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:14:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:14:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:14:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:14:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:17

In [170]:
params['max_depth'] = best_params

In [192]:
min_mae = float("Inf")
best_params = None
for min_child_weight in range(3,15):
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_params = xgb.cv(
        params,
        dtrain,
        num_boost_round=999,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_params['test-mae-mean'].min()
    boost_rounds = cv_params['test-mae-mean'].argmin()
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (min_child_weight)
print("Best params: {}, MAE: {}".format(best_params, min_mae))

[21:22:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:22:40] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:22:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:22:40] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language

[21:22:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:22:44] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:22:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:22:44] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language

[21:22:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:22:48] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:22:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:22:48] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language

[21:22:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:22:52] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:22:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:22:52] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language

Best params: 12, MAE: 16999.7185548


In [193]:
params['min_child_weight'] = best_params

In [198]:
min_mae = float("Inf")
best_params = None
for subsample_bytree in range(4,10):
    subsample_bytree = subsample_bytree/10.0
    print(subsample_bytree)
    params['subsample_bytree'] = subsample_bytree
    
   
    # Run CV
    cv_params = xgb.cv(
        params,
        dtrain,
        num_boost_round=999,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
     #Update best MAE
    mean_mae = cv_params['test-mae-mean'].min()
    boost_rounds = cv_params['test-mae-mean'].argmin()
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample_bytree)
print("Best params: {}, MAE: {}".format(best_params, min_mae))

0.4
[21:24:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:24:19] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:24:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:24:19] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in lang

0.7
[21:24:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:24:23] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:24:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:24:23] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in lang

Best params: 0.4, MAE: 16999.7185548


In [199]:
params['subsample_bytree'] = best_params


In [200]:
min_mae = float("Inf")
best_params = None
for colsample in range(3,10):
    colsample = colsample/10.
    params['colsample'] = colsample
    # Run CV
    cv_params = xgb.cv(
        params,
        dtrain,
        num_boost_round=999,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_params['test-mae-mean'].min()
    boost_rounds = cv_params['test-mae-mean'].argmin()
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (colsample)
print("Best params: {}, MAE: {}".format(best_params, min_mae))

[21:24:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:24:39] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:24:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:24:39] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language

[21:24:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:24:43] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:24:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:24:43] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language

[21:24:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:24:47] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:24:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:24:47] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language

In [201]:
params['colsample'] = best_params
params

{'max_depth': 10,
 'min_child_weight': 12,
 'subsample': 1,
 'colsample_bytree': 1,
 'objective': 'reg:linear',
 'subsample_bytree': 0.4,
 'colsample': 0.3}

In [202]:
mod_best = xgb.XGBRegressor(
    max_depth=params['max_depth'],                 
    min_child_weight=params['min_child_weight'],
    subsample = params['subsample'],
    colsample = params['colsample_bytree'],
    n_estimators=10000                                                                    
    
) 

In [203]:
mod_best.fit(X_train, y_train)
mod_best.score(X_train,y_train)

[21:25:00] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




0.9999999999996178

In [204]:
mod_best.score(X_test,y_test)

0.8422669958251519

# Undo Box Cox Transformation

In [ ]:
df["SalePrice"] = scipy.special.inv_boxcox(y, 0.25)
df.head()
#Undo both or y test only

In [ ]:
#Final model runs on transformed data here